In [13]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('chamados.db')

# Ler a tabela demanda no DataFrame do pandas
df = pd.read_sql_query("SELECT * FROM chamados", conn)

In [14]:
import sqlite3
from tkinter import *
from tkinter import messagebox, ttk, scrolledtext
from tkcalendar import DateEntry  # Importa o DateEntry do pacote tkcalendar
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Função para conectar ao banco de dados e criar a tabela se não existir
def connect_db():
    conn = sqlite3.connect('chamados.db')   
    cur = conn.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS chamados (
                    id INTEGER PRIMARY KEY,
                    descricao TEXT(500),
                    solicitante TEXT(100),
                    data_solicitacao TEXT,
                    numero_chamado TEXT,
                    como_foi_resolvido TEXT(500),
                    status TEXT)''')
    conn.commit()
    conn.close()

# Função para adicionar uma nova demanda ao banco
def add_demanda():
    descricao = descricao_entry.get("1.0", "end-1c")
    solicitante = solicitante_entry.get()
    data_solicitacao = data_entry.get_date().strftime('%d/%m/%Y')
    numero_chamado = numero_entry.get()
    como_foi_resolvido = como_foi_resolvido_entry.get("1.0", "end-1c")
    status = status_combobox.get()

    if descricao == "" or solicitante == "" or data_solicitacao == "" or status == "":
        messagebox.showerror("Erro", "Por favor, preencha todos os campos obrigatórios.")
        return  

    conn = sqlite3.connect('chamados.db')
    cur = conn.cursor()

    # Buscar o maior ID atual e definir o próximo ID
    cur.execute('SELECT MAX(id) FROM chamados')
    max_id = cur.fetchone()[0]
    next_id = (max_id + 1) if max_id else 1  # Se não houver registros, começa do 1

    # Inserir a nova demanda com o ID manualmente
    cur.execute('INSERT INTO chamados (id, descricao, solicitante, data_solicitacao, numero_chamado, como_foi_resolvido, status) VALUES (?, ?, ?, ?, ?, ?, ?)', 
                (next_id, descricao, solicitante, data_solicitacao, numero_chamado, como_foi_resolvido, status))

    conn.commit()
    conn.close()
    messagebox.showinfo("Sucesso", "Demanda adicionada com sucesso!")
    clear_entries()
    display_demandas()
        
# função para verificar demandas semelhantes
def verificar_similaridade():
    descricao = descricao_entry.get("1.0", "end-1c")

    if descricao.strip() == "":
        messagebox.showerror("Erro", "Por favor, insira uma descrição antes de verificar similaridade.")
        return

    # Conectar ao banco de dados e buscar as descrições existentes
    conn = sqlite3.connect('chamados.db')
    cur = conn.cursor()
    cur.execute("SELECT id, descricao, numero_chamado, status FROM chamados")
    rows = cur.fetchall()
    conn.close()

    if not rows:
        messagebox.showinfo("Info", "Nenhuma demanda cadastrada para comparação.")
        return

    # Criar DataFrame com os chamados existentes
    # Lista de solicitações
    solicitacoes = df['descricao'].tolist()

    # Nova solicitação recebida
    nova_solicitacao = descricao

    # Vetorização TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(solicitacoes + [nova_solicitacao])

    # Calcula similaridade
    similaridades = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

    # Converte para DataFrame para facilitar a visualização
    df_similaridade = pd.DataFrame(similaridades.flatten(), index=solicitacoes, columns=["Similaridade"])
    df_similaridade = df_similaridade.sort_values(by="Similaridade", ascending=False)

    sim = pd.DataFrame(similaridades.T)
    df['similaridade'] = sim

    df_sorted = df.sort_values(by='similaridade', ascending=False)
    df_selected_columns = df_sorted.iloc[:, [1,7,8]] # busca as variaveis descricao, como_foi_resolvido, similaridade
    # Exibir as 5 primeiras linhas
    #print(df_sorted.head(5))
        
    # Exibir as primeiras 5 linhas na interface
    update_displayed_df(df_selected_columns.head(5))
    
# Função para atualizar a exibição das 5 primeiras linhas
def update_displayed_df(df_head):
    # Limpar qualquer conteúdo anterior
    for widget in displayed_df_frame.winfo_children():
        widget.destroy()

    # Criar as colunas com cabeçalhos
    for col_idx, col_name in enumerate(df_head.columns):
        header_label = Label(displayed_df_frame, text=col_name, font=('Arial', 10, 'bold'))
        header_label.grid(row=0, column=col_idx, padx=5, pady=2)

    # Exibir os dados
    for row_idx, row_data in enumerate(df_head.values, start=1):
        for col_idx, (col_name, cell_value) in enumerate(zip(df_head.columns, row_data)):
            if col_name in ["descricao", "como_foi_resolvido"]:  # Se for uma dessas colunas, usar Text
                text_widget = Text(displayed_df_frame, font=('Arial', 10), width=60, height=3, wrap=WORD)
                text_widget.grid(row=row_idx, column=col_idx, padx=5, pady=2)
                text_widget.insert("1.0", str(cell_value))
                text_widget.config(state="disabled")  # Impede edição
            else:  # Para outras colunas, usar Entry normal
                cell_entry = Entry(displayed_df_frame, font=('Arial', 10), width=10)
                cell_entry.grid(row=row_idx, column=col_idx, padx=5, pady=2)
                cell_entry.insert(0, str(cell_value))
                cell_entry.config(state="readonly")  # Impede edição

# Função para limpar os campos de entrada
def clear_entries():
    descricao_entry.delete("1.0", END)
    solicitante_entry.delete(0, END)
    data_entry.set_date(datetime.today())  # Define a data atual no campo de data
    numero_entry.delete(0, END)
    como_foi_resolvido_entry.delete("1.0", END)
    status_combobox.set('')  # Limpa a seleção do Combobox

# Função para exibir as demandas em forma de tabela, filtradas por status
def display_demandas():
    conn = sqlite3.connect('chamados.db')
    cur = conn.cursor()
    
    # Seleciona apenas as colunas desejadas e filtra os status "verificar" e "aguardar ipm"
    cur.execute("""
        SELECT id, descricao, data_solicitacao, solicitante, numero_chamado, status 
        FROM chamados 
        WHERE status IN ('verificar', 'aguardar cliente', 'aguardar fábrica')
    """)
    rows = cur.fetchall()
    conn.close()

    # Limpar tabela existente, se houver
    for widget in scrollable_frame.winfo_children():
        widget.destroy()

    # Cabeçalho da tabela
    headers = ['ID','Descrição', 'Data', 'Solicitante', 'Número Chamado', 'Status']
    for col, header in enumerate(headers):
        header_label = Label(scrollable_frame, text=header, font=('Arial', 10, 'bold'), bg='lightgray', relief='ridge', width=20)
        header_label.grid(row=0, column=col, sticky='nsew')

    # Dados da tabela
    for row_idx, row_data in enumerate(rows, start=1):
        for col_idx, cell_value in enumerate(row_data):
            cell_label = Label(scrollable_frame, text=str(cell_value), font=('Arial', 10), wraplength=300, justify='left')
            cell_label.grid(row=row_idx, column=col_idx, sticky='nsew')

    # Atualiza a interface
    scrollable_frame.update_idletasks()
    scrollbar.config(command=canvas.yview)
    scrollable_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))


# Função para atualizar a exibição das demandas ao mudar o filtro de status
def update_filter(event):
    display_demandas()

# Interface gráfica
root = Tk()
root.title("Sistema de Controle de Demandas")

# Campos de entrada
Label(root, text="Descrição").grid(row=0, column=0, pady=5)
descricao_entry = scrolledtext.ScrolledText(root, height=5, width=100, wrap=WORD)
descricao_entry.grid(row=0, column=1, pady=5, padx=(10, 0))  # Define padx para a esquerda apenas

Label(root, text="Solicitante").grid(row=1, column=0, pady=5)
solicitante_entry = Entry(root, width=50)
solicitante_entry.grid(row=1, column=1, pady=5, padx=(10, 0))  # Define padx para a esquerda apenas

Label(root, text="Data de Solicitação").grid(row=2, column=0, pady=5)
data_entry = DateEntry(root, width=12, background='darkblue', foreground='white', borderwidth=2, date_pattern='dd/mm/yyyy')
data_entry.grid(row=2, column=1, pady=5, padx=(10, 0))  # Define padx para a esquerda apenas

Label(root, text="Número do Chamado").grid(row=6, column=0, pady=5)
numero_entry = Entry(root, width=20)
numero_entry.grid(row=6, column=1, pady=5, padx=(10, 0))  # Define padx para a esquerda apenas

Label(root, text="Como foi resolvido").grid(row=7, column=0, pady=5)
como_foi_resolvido_entry = scrolledtext.ScrolledText(root, height=5, width=100, wrap=WORD)
como_foi_resolvido_entry.grid(row=7, column=1, pady=5, padx=(10, 0))  # Define padx para a esquerda apenas

Label(root, text="Status").grid(row=5, column=0, pady=5)
status_combobox = ttk.Combobox(root, values=['verificar', 'aguardar cliente', 'aguardar fábrica', 'finalizado'], width=17)
status_combobox.grid(row=5, column=1, pady=5, padx=(10, 0))  # Define padx para a esquerda apenas



# Botões
Button(root, text="Verificar", command=verificar_similaridade, bg="yellow").grid(row=3, column=0, columnspan=2, pady=10)
Button(root, text="Adicionar Demanda", command=add_demanda).grid(row=8, column=0, columnspan=2, pady=10)

# Filtro por status # Caso queira habilitar a função para mostrar todas as demandas
#Label(root, text="Filtrar por Status:").grid(row=9, column=0, pady=5)
#status_filter = ttk.Combobox(root, values=['Todos', 'verificar', 'aguardar cliente', 'aguardar fábrica', 'finalizado'], width=17)
#status_filter.current(0)  # Define 'Todos' como padrão
#status_filter.grid(row=9, column=1, pady=5, padx=(10, 0))
#status_filter.bind("<<ComboboxSelected>>", update_filter)

# Configuração da grade para expandir conforme o tamanho da janela
root.grid_rowconfigure(8, weight=1)
root.grid_columnconfigure(1, weight=1)  # Define a segunda coluna (coluna 1) como expansível

# Conectar ao banco de dados e criar a tabela se não existir
connect_db()

 # Adicionar o frame para exibir as 5 primeiras linhas
displayed_df_frame = Frame(root)
displayed_df_frame.grid(row=4, column=0, columnspan=2, pady=10)  # Agora abaixo do botão "Adicionar Demanda"
                            
                            
# Frame para exibir a tabela de demandas com scrollbar - (mostrar as demandas)
canvas = Canvas(root)
scrollbar = Scrollbar(root, orient="vertical", command=canvas.yview)
scrollable_frame = Frame(canvas)

scrollable_frame.bind(
    "<Configure>",
    lambda e: canvas.configure(
        scrollregion=canvas.bbox("all")
    )
)

canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
canvas.configure(yscrollcommand=scrollbar.set)

canvas.grid(row=10, column=0, columnspan=2, sticky="nsew")
scrollbar.grid(row=10, column=2, sticky="ns")

# Exibir as demandas inicialmente
display_demandas()

# Iniciar a interface gráfica
root.mainloop()



Consultar o banco de dados

In [4]:
import pandas as pd
conn = sqlite3.connect('chamados.db')

# Ler a tabela demanda no DataFrame do pandas
df = pd.read_sql_query("SELECT * FROM chamados", conn)

In [5]:
import sqlite3
import pandas as pd

# Conectar ao banco de dados
conn = sqlite3.connect('chamados.db')

# Ler a tabela demanda no DataFrame do pandas
df = pd.read_sql_query("SELECT * FROM chamados", conn)

# Fechar a conexão com o banco de dados
conn.close()

# Exibir o conteúdo da tabela demanda
print(df)

## df.to_excel('atividades.xlsx', index=False)

        id                                          descricao solicitante  \
0        1      Erro ao estornar parcelamento - erro de saldo      Carmen   
1        2              Erro ao desenglobar - erro de tributo      Carmen   
2        3     corrigir o relatório de compensação de valores      Flavio   
3        4  Ajustar layout do carnê de parcelamento para  ...     Marlene   
4        5  Alterar data de vencimento de um lançamento tr...       Carol   
...    ...                                                ...         ...   
1161  1162  Como tira aquele relatório que mostra o livro ...      Josias   
1162  1163  Sub-receita 69 esta utilizando uma forma de pa...      Sandro   
1163  1164  Fazer ajustes nas formas de pagamento 84 para ...   Miriam VS   
1164  1165  Verificar motivo pelo qual o valor aumentou qu...   Maristela   
1165  1166              Incluir sevidores em centro de custos   Maristela   

         data_solicitacao numero_chamado observacao      status  \
0     20

In [8]:
df.tail(25)

,id,descricao,solicitante,data_solicitacao,numero_chamado,observacao,status,como_foi_resolvido
1141,1142,Fazer relatório de parcelamento a receber por mês,Joacir,17/02/2025,None,None,finalizado,Para isso devemos gerar um relatio de gerencia...
1142,1143,Verificar o porque não esta aparecendo o nome ...,Janice,17/02/2025,None,None,finalizado,Para que possa ser tramitado no caso da procur...
1143,1144,"Tem como fazer uma averbação em um endereço, s...",Miriam,17/02/2025,None,None,finalizado,"Não tem como, precisa ser um cadastro imobilia..."
1144,1145,Remover o serviço de 'comprovante de isenção 2...,Jaqueline,17/02/2025,None,None,finalizado,"Portal do Cidadão » Gerenciar » Serviços, pesq..."
1145,1146,Corrigir layouts de certidão negativa - inform...,Ana portela,17/02/2025,None,None,verificar,alterar via birt
1146,1147,Ver erro que ocorre ao excluir relacionamento ...,Tamara,17/02/2025,None,None,verificar,ver a possibilidade de alterar via banco
1147,1148,verificar por que não bate os valores do tribu...,Nelci,17/02/2025,756472,None,aguardar fábrica,None
1148,1149,Ver sobre a utilização do campo de prescrição ...,Evandro,17/02/2025,None,None,verificar,conversar com o evandro
1149,1150,O contribuinte logado não consegue clicar na l...,Cledimara,2025-02-18 00:00:00,None,None,finalizado,o serviço em questão estava aceitando pessoas ...
1150,1151,criar um serviço para o contribuinte imprimir ...,Cledimara,2025-02-18 00:00:00,None,None,finalizado,foi ativado o serviço 109 e criado um layout d...


Alterar informação no banco de dados

In [3]:
from tkinter import *
from tkinter import messagebox


# Função para conectar ao banco de dados e realizar uma alteração
def alterar_demanda(id_demanda, novo_status):
    try:
        conn = sqlite3.connect('chamados.db')
        cur = conn.cursor()
        cur.execute('UPDATE chamados SET como_foi_resolvido = ? WHERE id = ?', (novo_status, id_demanda)) # depois do SET incluir a variavel que pretende atualizar
        conn.commit()
        conn.close()
        messagebox.showinfo("Sucesso", f"Demanda ID {id_demanda} alterada com sucesso!")
    except sqlite3.Error as e:
        messagebox.showerror("Erro", f"Erro ao alterar demanda: {e}")

# Exemplo de uso da função para alterar uma demanda específica
# Supondo que você tenha o ID da demanda e o novo status que deseja atribuir
id_demanda = 1137  # Substitua pelo ID da demanda que deseja alterar
novo_status = 'teste'  # Novo status que deseja atribuir

alterar_demanda(id_demanda, novo_status)